<a href="https://colab.research.google.com/github/saurabhkirar/EPAI01/blob/main/Session6/6B_Decorators/Decorators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Declare the variables and the libraries that will be shared

In [48]:
from functools import wraps
from datetime import datetime
logger_dict=dict()
from  time import perf_counter

This is the notebook for decorators, we will write the decortors for the following

Q1 allows a function to run only on odd seconds 

In [ ]:
def func_factory_outer(chk):
  '''This is an outer enclosure as we cant pass any argument to the closure'''
  def deco_odd_fn(fn):
    '''This is the outer function which will be used by the calling 
    environment'''
    @wraps(fn)
    def inner(*args,**kwargs):
      ''' This is the inner function which will be executed'''
      res=fn(*args,**kwargs) if chk==True else 'Error,cant run the function'
      '''Run the function with paramenters if the check is pass else throw an error'''
      return res
    return inner
  return deco_odd_fn 

In [ ]:
    ''' Lets test this.'''
sec=datetime.now().second
chk=False if sec%2==0 else True
print(chk)
@func_factory_outer(chk)
def func_add(*args,**kwrags):
  return sum(args)

func_add(8,5)

True


13

Q2 - Write a decorator to log the function

In [ ]:
'''This is the outer function that will capture the logging details'''
def func_logger(fn):
  cnt=0
  '''The inner function is a closure'''
  @wraps(fn)
  def inner_fn(*args,**kwargs):
    '''use non local to acccess and change a variable not defined within the function scope'''
    nonlocal cnt 
    cnt+=1
    fn_name=fn.__name__
    fn_args=args
    fn_cnt=cnt
    '''This returns the dictionary to track'''
    logger_dict[(fn_name,fn_args)]=cnt
    return fn(*args,**kwargs) ## this means that the actual function will run.

  return inner_fn



In [ ]:
@func_logger
def func_add(*args,**kwargs):
  return sum(args)

In [ ]:
'''The below code will test the function and a logger dictionary is printed
Key is a tuple of function name and the agrs where as the value is the count of the function call'''
func_add(4,5),func_add(87,6)
print(logger_dict)

{('func_add', (4, 5)): 9, ('func_add', (87, 6)): 10}


Q3 - Write a decorator to authenticate  the user, here assumption is taken for the domain name as gmail.com which means any user who doesnt have the domain addressed should not be allowed to do a specific task.
The task is to add two numbers.

In [1]:
'''lets define an outer function that takes a truth value and this truth value will be preculated to the further elements of the code'''
def check_user_auth(chk):
  '''lets define a decorator now'''
  def func_user_auth(fn):
    '''below is the actual function that would be executed'''
    def inner_fn(*args,**kwargs):
      return fn(*args,**kwargs) if chk==True else 'You are not authorized.'
      '''the below returns the closure'''
    return inner_fn
  '''The below returns the decorator'''
  return func_user_auth

In [31]:
'''now lets test the function'''
user_names=['ash@gmail.com','gmail@gmail.com','gmail@yahoo.com','imsrk@srk.com']
truth_condition=list((map(fn,user_names)))

In [38]:
'''now call the decorator factory function and initiate with the truth condition'''
for condition in truth_condition:
  @check_user_auth(condition)
  def test_add(*args,**kwargs):
    return sum(args)

  print('calling for ',condition)
  print(test_add(8,7))

calling for  True
15
calling for  True
15
calling for  False
You are not authorized.
calling for  False
You are not authorized.


Q4 Write a decorator to time a function.

In [55]:
'''The below code shows a decorator to time a function, but any function that needs to be evaluated 
must be evaluated over few iterations and not just the one so we will have a user flexibility for cross calidation 
and the resut would be the mean time of all the iterations'''
def timed(num_reps):
   def deco_timed(fn):
   
    def inner(*args, **kwargs):

        '''This is the function which will run value equal to num_reps'''

        total_elapsed = 0
        for i in range(num_reps):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / num_reps
        print('Avg Run time: {0:.6f}s ({1} reps)'.format(avg_elapsed,
                                                        num_reps))
        return result
    
    return inner

   return deco_timed


In [49]:
@timed(5)
def test_add(*args,**kwargs):
    return sum(args)

In [50]:
test_add(8,6)

Avg Run time: 0.000001s (5 reps)


14

Q5. Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params)

In [223]:

def validate_access(p_priv):
  def func_arith_deco(fn):
    lst=['dummy']
    l_append=False
    l_modify=False
    l_delete=False
    def func_arith_sum(*args,**kwargs):
      '''get the variables initialized above'''
      nonlocal l_append
      nonlocal l_modify
      nonlocal l_delete
      val=[key for key in p_priv.keys()]
      print(val[0])
      if val[0]=='high':
        l_append=True
        l_modify=True
        l_delete=True
      elif val[0]=='mid':
        l_append=True
        l_modify=True
      elif val[0]=='low':
        l_append=True
      elif val[0]=='no':
        l_dummy=True

      '''does the append'''
      if l_append:
        lst.append(fn.__name__) 
      else :
        print('You are not authorize to add anything...')

      '''modify the list'''
      if l_modify:

        lst[0]='modified'
      else :
        print('You are not authorize to modify  anything...')

      '''delete the list'''
      if l_delete:
        lst.remove(max(lst))
        print('You are a super user..')
      else :
        print('You are not authorize to modify  anything...')

     # print(lst)
      '''reset the flags'''
      l_append=False
      l_modify=False
      l_delete=False
      return lst
    
    return func_arith_sum

  return func_arith_deco




In [232]:
'''
1. Make a list of all the operations and the privilages
2. Make a pair of priv and the permissible operations
3. decorate the test_add function with the facotry function @validate access
4. in @validate access pass the pair
'''
access_list=['Delete','Modify','Add','read only']
priv=['high','mid','low','no']
pair=dict()
for i in range(len(access_list)):
  pair[priv[i]]=access_list[i:]
a,b=next(iter(pair.items()))
@validate_access({a:b})
def test_add(*args):
  return sum(args)



In [233]:
'''Lets test by calling the function'''
test_add(8,6)

high
You are a super user..


['modified']